In [ ]:
import serial_asyncio
import time 
import asyncio
import queue

In [ ]:
class SerialWorker():
    def __init__(self, port, baudrate, loop):
        self.port = port
        self.baudrate = baudrate
        self.loop = loop
        self._queue = asyncio.Queue()
        self._reader = None
        self._writer = None
        self.to_device = asyncio.Queue()
        self.busy = False

    def start(self):
        self.loop.create_task(self._connect())
        self.loop.create_task(self._serial_loop())

    async def _connect(self):
        self._reader, self._writer = await serial_asyncio.open_serial_connection(
            url=self.port, baudrate=self.baudrate, loop=self.loop
        )

    def is_connected(self):
        return self._reader is not None and self._writer is not None
    async def _serial_loop(self):
        while True:
            #check if the printer is connected
            if self.is_connected():
                if not self.busy:
                    data = await self.to_device.get()
                # send command
                data = data + '\r'
                print('sending: ', data)
                self._writer.write(data.encode())
                await self._writer.drain()
                data = await self._reader.readuntil()
                print('received: ', data)
                self._queue.put_nowait(data)
                if b'ok' in data:
                    self.busy = False
                    if self.to_device.empty():
                        self.to_device.task_done()
                if b'echo:busy: processing' in data:
                    self.busy = True


In [ ]:
worker = SerialWorker(port='COM3', baudrate=115200, loop=asyncio.get_event_loop())
worker.start()

In [ ]:
positions = [0.0, 100.0]
iterations = 100
counter = 0

while counter < iterations:
    position = positions[counter % 2]
    worker.to_device.put_nowait('G1 X{}'.format(position))
    print('sent: ', position)
    await worker.to_device.join()
    print('done: ', position)
    counter += 1
